# Notebook to compute polynomial fits
Written by Anne Zarda (anne.zarda@epfl.ch)

In [ ]:
! pip install seaborn

In [15]:
from math import sqrt
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures

ROOT_PATH = Path.cwd().parent
TEST_RANDOM_DW = ROOT_PATH / "data" / "perm_random20_test_dw.csv"
TRAIN_RANDOM_DW = ROOT_PATH / "data" / "perm_random80_train_dw.csv"

In [ ]:
## cross validation to determine which polynomial fit should be used

df = pd.read_csv(TRAIN_RANDOM_DW)
# rename column
df = df.rename(columns={"target": "CAPA [1 µM]"})

savedir = Path(ROOT_PATH, "figures")
savedir.mkdir(exist_ok=True)

xes = [
    "MW",
    "cLogP",
    "cLogS",
    "HBA",
    "HBD",
    "Total Surface Area",
    "Rel. PSA",
    "PSA",
    "Rot. Bonds",
    "Amides",
]  # properties that are fit against permeability (will be x in the fitting)

dfest = pd.DataFrame(columns=["type"] + [i for i in range(5)])  # dataframe to collect estimators

dfr2train = pd.DataFrame(
    columns=["type"] + [i for i in range(5)]
)  # dataframe to collect r2 scores on training set

dfr2validation = pd.DataFrame(
    columns=["type"] + [i for i in range(5)]
)  # dataframe to collect r2 scores on validation set

for thing in xes:
    x_raw = df[thing]
    y_raw = df["CAPA [1 µM]"]

    x = df[thing].array.reshape(-1, 1)
    y = df["CAPA [1 µM]"].array.reshape(-1, 1)

    xmin = min(x_raw) - 0.1 * (max(x_raw) - min(x_raw))
    xmax = max(x_raw) + 0.1 * (max(x_raw) - min(x_raw))

    xp = np.arange(xmin, xmax, (xmax - xmin) / 100).reshape(-1, 1)  # a dummy array to plot the fits

    # print(x.shape)
    # print(y.shape)

    results = {}

    r2s_validation = []  # list to collect r2 scores on validation set
    r2s_train = []  # list to collect r2 scores on training set
    estimators = []  # list to collect estimators

    colors = sns.color_palette("YlGn", n_colors=12, as_cmap=False)

    fig, axn = plt.subplots(1, 5, figsize=(25, 5))

    for d in range(5):
        # print('Polynomial degree: ' + str(d))
        x_poly = PolynomialFeatures(degree=d).fit_transform(x)  # Featurization of x
        lr = LinearRegression(fit_intercept=False)  # linear regression model
        model = lr.fit(x_poly, y)  # fit linear regression

        estimators.append(model)  # save fit to list

        # display(model.coef_)
        # display(model.intercept_)

        # # cross validation of fit
        cvs = 10
        cve = cross_validate(
            lr,
            x_poly,
            y,
            cv=cvs,
            scoring="r2",
            return_train_score=True,
            return_estimator=True,
        )
        r2s_validation.append(np.mean(cve["test_score"]))  # save mean r2 scores from validation set
        r2s_train.append(np.mean(cve["train_score"]))  # save mean r2 scores from train set
        # display(cve)

        if (
            d % 1 == 0
        ):  # this can be adjusted if not all fits should be plotted (for example only every second, third, fourth, ...)
            for cvidx in range(cvs):
                axn.flat[d].plot(
                    xp,
                    cve["estimator"][cvidx].predict(PolynomialFeatures(degree=d).fit_transform(xp)),
                    "-",
                    color=colors[cvidx],
                )  # plot each fit from the cross validation
            axn.flat[d].plot(
                xp,
                model.predict(PolynomialFeatures(degree=d).fit_transform(xp)),
                "-",
                color="black",
            )  # plot fit from whole training set
            axn.flat[d].plot(x_raw, y_raw, ".")  # plot experimental values
            axn.flat[d].set_ylim(-10, 110)
            axn.flat[d].set_xlim(xmin, xmax)
            axn.flat[d].set_xlabel(thing)
            axn.flat[d].set_ylabel("%-HaloTag labeling")
            axn.flat[d].set_title("Polynomial fitting: " + str(d))

    fig.suptitle(thing, y=1.005, fontsize=20)
    fig.tight_layout()
    plt.savefig(Path(savedir, f"cv_{thing}.jpg"), bbox_inches="tight", dpi=400)
    plt.show()

    # plot of mean r2 scores for validation set (blue) and training set (green)
    fig, axn = plt.subplots(1, 1, figsize=(3, 3))
    axn.plot(
        range(len(r2s_validation)),
        r2s_validation,
        ".",
        label="validation r2 score",
        alpha=0.5,
        color="blue",
    )
    axn.plot(
        range(len(r2s_validation)),
        r2s_validation,
        "-",
        label="validation r2 score",
        alpha=0.5,
        color="blue",
    )
    axn.plot(
        range(len(r2s_train)),
        r2s_train,
        ".",
        label="train r2 score",
        alpha=0.5,
        color="green",
    )
    axn.plot(
        range(len(r2s_train)),
        r2s_train,
        "-",
        label="train r2 score",
        alpha=0.5,
        color="green",
    )
    axn.set_xlabel("Polynomial degree")
    axn.set_ylabel("r2 score")
    axn.set_title(thing)
    axn.set_ylim(-100, 10)
    axn.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    # plt.savefig(Path(savedir, f'cv_scores_{thing}.jpg'), bbox_inches='tight', dpi=400)
    plt.show()

    dfest.loc[len(dfest.index)] = [thing] + estimators  # save estimators
    dfr2train.loc[len(dfr2train.index)] = [
        thing
    ] + r2s_train  # save mean r2 scores from training set
    dfr2validation.loc[len(dfr2validation.index)] = [
        thing
    ] + r2s_validation  # save mean r2 scores from validation set

In [17]:
dfest.set_index("type", inplace=True)
dfr2validation.set_index("type", inplace=True)
dfr2train.set_index("type", inplace=True)
# display(dfest, dfr2validation, dfr2train)

In [ ]:
# write all polynomial estimators to excel file

resultpath = Path(ROOT_PATH, "results")
resultpath.mkdir(exist_ok=True)

dfcoef = pd.DataFrame(columns=["type", "degree"] + [i for i in range(5)])  # create empty dataframe

rows = dfest.index.values.tolist()
cols = [i for i in dfest]

for row in rows:
    # print(row)
    for col in cols:
        # print(col)
        coeffs = [i for i in dfest.loc[row, col].coef_.tolist()[0]]
        # print(coeffs)
        newrow = [row] + [col] + coeffs + [0 for i in range(5 - len(coeffs))]
        # print(newrow)
        dfcoef.loc[len(dfcoef.index)] = newrow
        # display(dfcoef)

display(dfcoef)
dfcoef.to_csv(Path(resultpath, "coefs.csv"))

In [ ]:
# plot heatmap with mean r2 validation test scores

plt.subplots(1, 1, figsize=(7, 5))
sns.heatmap(data=dfr2validation, vmin=-5, vmax=0, square=True, annot=True)
plt.title("mean r2 scores from cross validation")
plt.xlabel("polynomial degree")
plt.savefig(Path(resultpath, f"cvevalidationr2score.jpg"), bbox_inches="tight", dpi=600)
dfr2validation.to_csv(Path(resultpath, "dfr2validation.csv"))

In [ ]:
# find best polynomial degree for each fit

bestvalidation = dfr2validation.idxmax(axis=1)

print(bestvalidation)

bestfit_dict = {}  # dictionary to save best estimator as determined by cross validation

for thing in bestvalidation.index:
    print(thing)
    display(dfest.loc[thing, bestvalidation[thing]].coef_)
    bestfit_dict[thing] = dfest.loc[thing, bestvalidation[thing]]  # save estimator to dictionary

In [ ]:
# Testing

# import test set
df_test = pd.read_csv(TEST_RANDOM_DW)
# rename column
df_test = df_test.rename(columns={"target": "CAPA [1 µM]"})
df_test.head()

In [ ]:
# using the extra 20 % of the data for final evaluation of the fits

y20 = df_test["CAPA [1 µM]"]
dfpred = pd.DataFrame(data={"y_obs": y20})  # create dataframe to save predictions

for thing in xes:
    print(thing)
    x20 = df_test[thing].array.reshape(-1, 1)

    bestmodel = bestfit_dict[thing]

    degree = len(bestmodel.coef_[0]) - 1
    print("degree: " + str(degree))

    x20_poly = PolynomialFeatures(degree=degree).fit_transform(x20)
    y20_pred = bestmodel.predict(x20_poly)
    dfpred["pred_" + str(thing)] = y20_pred
    print("r2_score: " + str(r2_score(y20, y20_pred)))
    print("rmse: " + str(sqrt(mean_squared_error(y20, y20_pred))))
    print("---------")

display(dfpred)
dfpred.to_csv(Path(resultpath, "predictions20.csv"))